# imports

In [10]:
'modified by abdul alkurdi; 10/05/2023'

import pandas as pd
#import cudf
import pickle
import json
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib import figure
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.signal import correlate
#import cupy as cp
from scipy.io import wavfile
#from scipy import signal, stats
#import peakutils, wfdb, pywt
import csv
import os, statistics
from datetime import datetime
#import heartpy as hp
import json
%matplotlib notebook
# import neurokit2 as nk

import process_redcap 
from syncfcns import *

# inits

In [11]:
################################
#####   force update ?     #####
################################
force_update = False


radwear_path = '/projects/bbnp/Vansh/data/RADWear/'
redcap_path = radwear_path+'REDCap responses/'

# load all participant meta data
with open(radwear_path+'all_p_metadata.json', 'rb') as f:
            all_p_metadata = json.load(f)
# load all participant redcap data
redcap_df = process_redcap.process_redcap(redcap_path,all_p_metadata['list of participant IDs'])
list_of_participants = all_p_metadata['list of participant IDs']
completed_participants = []


# label definitions
calib_dict = {'meditation': 0, 'cpt': 1}
rot_anx_dict = {'calibration': 0, 'LA': 1, 'HA': 2}

p_calib = {}
p_la = {}
p_ha = {}
p_all = {} 
incomplete = [guy for guy in list_of_participants if guy not in completed_participants]

redcap dict exists
redcap df exists
redcap df pickle loaded


# fcn defs

In [13]:
def read_sync_return(filepath, e4_today, hx_today):
    '''
    usage: read_sync_return(filepath, e4_today, hx_today)
    returns: synced_participant_data
    '''
    e4_dict = read_E4(filepath+'/'+e4_today, hx_today)
    hx_dict = read_hx(filepath+'/record_'+hx_today, hx_today)

    #e4sync_offset = E4sync_offset(e4_dict)
    #hxsync_offset = Hexsync_offset(hx_dict['ECG'],hx_dict['BR'],hx_dict['accx'],hx_dict['accy'], hx_dict['accz'])
    e4sync_offset = e4_dict
    hxsync_offset = hx_dict
    
    synced_participant_data = {}
    #cross syncing between devices but only on bvp and ecg
    synced_participant_data['ECG'], synced_participant_data['BVP'] = doublesync_offset(hxsync_offset['ECG'], e4sync_offset['BVP'])
    # for some reason we're losing seconds from beginning and end of hx data even though e4 is contained within it's timeframe.


    # running syncing on rest of signals for hx (ECG, breathing_rateacc, br)
    t0 = synced_participant_data['ECG']['Second'].iloc[0] #hx
    tf = synced_participant_data['ECG']['Second'].iloc[-1] #hx
    synced_participant_data['BR'] = hxsync_offset['BR'].loc[hxsync_offset['BR']['Second'] >= t0 ].loc[hxsync_offset['BR']['Second'] <= tf ]
    del tf, t0

    # running syncing on rest of signals for e4 (acc, eda, temp, hr, ibi )
    t0 = synced_participant_data['BVP']['Second'].iloc[0] #e4
    tf = synced_participant_data['BVP']['Second'].iloc[-1] #e4
    synced_participant_data['TEMP'] = e4sync_offset['TEMP'].loc[e4sync_offset['TEMP']['Second'] >= t0 ].loc[e4sync_offset['TEMP']['Second'] <= tf ]
    synced_participant_data['EDA'] = e4sync_offset['EDA'].loc[e4sync_offset['EDA']['Second'] >= t0 ].loc[e4sync_offset['EDA']['Second'] <= tf ]
    synced_participant_data['HR'] = e4_dict['HR'].loc[e4_dict['HR']['Second'] >= t0 ].loc[e4_dict['HR']['Second'] <= tf ]
    #synced_participant_data['IBI'] = e4sync_offset['IBI'].loc[e4sync_offset['IBI']['Second'] >= t0 ].loc[e4sync_offset['IBI']['Second'] <= tf ]
    del tf, t0

    hx_dict['ACC'] = hx_dict['accx']
    hx_dict['ACC']['Acc_Y'] = hx_dict['accy']['Acc_Y']
    hx_dict['ACC']['Acc_Z'] = hx_dict['accz']['Acc_Z']
     
    acc_dic = accsync_offset(e4sync_offset['ACC'],hxsync_offset['accx'])
    #offset_ecg_cross, offset_bvp_cross = doublesync_offset(e4sync_offset['ECG'], e4sync_offset['BVP'])
    synced_participant_data['ACC_hx'] = acc_dic['acc_hx']
    synced_participant_data['ACC_e4'] = acc_dic['acc_e4']

    return synced_participant_data
    


In [14]:
#########################
# p5 HA day 8 and p9 HA day 10 will be done manually. redcap is marked as 0 but data is available
#########################

# wesad.keys() = dict_keys(['signal', 'label', 'subject']); label returns array subject: S$ 
incomplete = [guy for guy in list_of_participants if guy not in completed_participants]

for p in list_of_participants:
    # check if participant pickle file exists
    if Path(radwear_path+'Participant '+str(p)+'/p_'+str(p)+'.pkl').is_file():
        print(radwear_path+'Participant '+str(p)+'/p_'+str(p)+'.pkl')
        print('participant ', p, ' already processed.')
        completed_participants.append(p)
incomplete = [guy for guy in list_of_participants if guy not in completed_participants]
if force_update:
    incomplete = list_of_participants



/projects/bbnp/Vansh/data/RADWear/Participant 4/p_4.pkl
participant  4  already processed.
/projects/bbnp/Vansh/data/RADWear/Participant 7/p_7.pkl
participant  7  already processed.
/projects/bbnp/Vansh/data/RADWear/Participant 9/p_9.pkl
participant  9  already processed.
/projects/bbnp/Vansh/data/RADWear/Participant 12/p_12.pkl
participant  12  already processed.
/projects/bbnp/Vansh/data/RADWear/Participant 14/p_14.pkl
participant  14  already processed.
/projects/bbnp/Vansh/data/RADWear/Participant 16/p_16.pkl
participant  16  already processed.
/projects/bbnp/Vansh/data/RADWear/Participant 17/p_17.pkl
participant  17  already processed.
/projects/bbnp/Vansh/data/RADWear/Participant 18/p_18.pkl
participant  18  already processed.
/projects/bbnp/Vansh/data/RADWear/Participant 21/p_21.pkl
participant  21  already processed.


In [16]:
for p in incomplete:
    print('procesing participant: ', p, ' calibration data...')
    p_all[p] = {}
    p_la[p] = {}
    p_ha[p] = {}
    p_path = radwear_path+'Participant '+str(p)
    # load participant e4 data
    e4sn = all_p_metadata[str(p)]['e4sn']
    calibration_files = all_p_metadata[str(p)]['calibration']
    LA = all_p_metadata[str(p)]['LA']
    HA = all_p_metadata[str(p)]['HA']
    
    # load calibration data
    e4_num = all_p_metadata[str(p)]['e4sn']+'_'+all_p_metadata[str(p)]['calibration'][0]
    hx_num = str(all_p_metadata[str(p)]['calibration'][1])
    p_calib[p] = read_sync_return(p_path, e4_num, hx_num)
    p_calib[p]['rot_label'] = rot_anx_dict['calibration'] * np.ones(len(p_calib[p]['ECG'])) # add label to designate calibration segment
    
    #####################
    # for when adding labels for cpt and meditation
    #calib_label = for 
    
    if all_p_metadata[str(p)]['complete days'][0] > 0:
        # loop for LA
        for i in range(all_p_metadata[str(p)]['complete days'][0]): 
            if not (all_p_metadata[str(p)]['RedCap available'][0][i] == 0 or all_p_metadata[str(p)]['LA'][0][i] == 0 or all_p_metadata[str(p)]['LA'][1][i] == 0): 
                print('procesing participant: ', p, ' LA day ', i+1, '...')    
                e4_num = all_p_metadata[str(p)]['e4sn']+'_'+all_p_metadata[str(p)]['LA'][0][i]
                hx_num = str(all_p_metadata[str(p)]['LA'][1][i])
                
                p_la[p][i] = read_sync_return(p_path, e4_num, hx_num)
                p_la[p][i]['rot_label']= rot_anx_dict['LA'] * np.ones(len(p_la[p][i]['ECG'])) 
            else:
                p_la[p][i] = {} 
    if all_p_metadata[str(p)]['complete days'][1] > 0:
        # loop for HA    
        for j in range(all_p_metadata[str(p)]['complete days'][1]): # loop for HA
            if not (all_p_metadata[str(p)]['RedCap available'][1][j] == 0 or all_p_metadata[str(p)]['HA'][0][j] == 0 or all_p_metadata[str(p)]['HA'][1][j] == 0): 
                print('procesing participant: ', p, ' HA day ', j+1, '...')    
                e4_num = all_p_metadata[str(p)]['e4sn']+'_'+all_p_metadata[str(p)]['HA'][0][j]
                hx_num = str(all_p_metadata[str(p)]['HA'][1][j])
                
                p_ha[p][j] = read_sync_return(p_path, e4_num, hx_num)
                p_ha[p][j]['rot_label']= rot_anx_dict['HA'] * np.ones(len(p_ha[p][j]['ECG']))
            else:
                p_ha[p][j] = {}
    
    print('participant ', p, ' done.')
    if True:
        try:
            p_all[p]['calib']= p_calib[p]
        except:
            print('participant ', p, ' has no calibration data.')
            p_all[p]['calib'] = {}
        try:    
            p_all[p]['LA'] = p_la[p]
        except:
            print('participant ', p, ' has no LA data.')
            p_all[p]['LA'] = {}      
        try: 
            p_all[p]['HA'] = p_ha[p]
        except:
            print('participant ', p, ' has no HA data.')
            p_all[p]['HA'] = {}
        #save this participant data to a pickle file
        
    with open(p_path+'/p_'+str(p)+'.pkl', 'wb') as handle:
            pickle.dump(p_all[p], handle, protocol=pickle.HIGHEST_PROTOCOL)
    #p_all[p].to_parquet('/p_'+str(p)+'.parquet', engine='pyarrow')

    print('participant ', p, ' pickled.')

    #df = pd.DataFrame.from_dict(p_all[p])
    #df.to_parquet(p_path+'/p_'+str(p)+'.parquet')
    #print('participant ', p, ' parqued.')

        

    completed_participants.append(p)
    print('-----------------------------------')

print('completed participants: ', completed_participants)
# print('incomplete participants: ', incomplete) doesn't update after loop completes them. 

procesing participant:  12  calibration data...
procesing participant:  12  LA day  1 ...
procesing participant:  12  LA day  2 ...
procesing participant:  12  LA day  3 ...
procesing participant:  12  LA day  4 ...
procesing participant:  12  LA day  5 ...
procesing participant:  12  LA day  6 ...
procesing participant:  12  LA day  7 ...
procesing participant:  12  LA day  8 ...
procesing participant:  12  HA day  1 ...
procesing participant:  12  HA day  2 ...
procesing participant:  12  HA day  3 ...
procesing participant:  12  HA day  4 ...
procesing participant:  12  HA day  5 ...
procesing participant:  12  HA day  6 ...
procesing participant:  12  HA day  7 ...
procesing participant:  12  HA day  8 ...
procesing participant:  12  HA day  9 ...
procesing participant:  12  HA day  10 ...
participant  12  done.
participant  12  pickled.
-----------------------------------
completed participants:  [4, 7, 9, 12, 14, 16, 17, 18, 21, 12]
incomplete participants:  [12]


In [17]:
p_path+'/p_'+str(p)+'.pkl'

'/projects/bbnp/Vansh/data/RADWear/Participant 12/p_12.pkl'

# formatting data

In [ ]:
#all_p = {}
#load each participant's data
force_update = False
for p in list_of_participants:
    p_path = radwear_path + 'Participant '+str(p)
    p_df = pd.DataFrame()
    
    #check if file exist
    a = 'available.' if os.path.isfile(p_path + '/p_'+str(p)+'.pkl') else ' not available.'
    print('pickle file for participant '+str(p)+ ' is '+ a)
    
    #all_p[p] = p_data ## this takes too much memory so i will just load each p when needed
    if not (os.path.isfile(radwear_path +'p_'+str(p)+'.pkl')) or force_update:


        with open(p_path +  '/p_'+str(p)+'.pkl', 'rb') as f:
            p_data = pickle.load(f)
        print ('participant '+str(p)+ ' data loaded.')
        print('-----------------------') 
        print('Participant: ', p)
        print('-----------------------')    
        #print('p_data.keys(): ', p_data.keys())
        #print('-----------------------')
        print('la days: ', len(p_data['HA'].keys()))
        print('ha days: ', len(p_data['LA'].keys()))
        print('-----------------------')
        #print('p_data[calib].keys(): ', p_data['calib'].keys())
        # e4 contains BVP, EDA, TEMP, ACC, IBI, HR, HRV, tags
        # hx contains ECG, ACC, BR
        p_redcap = redcap_df.loc[redcap_df['participant'] == p] # redcap data for participant p


        ## this is for participant p 

        for day in list(p_data['LA'].keys()): 

            proceed1 = bool(len(p_data['LA'][day])) and bool(all_p_metadata[str(p)]['RedCap available'][0][day])
            
            if proceed1:
                
                LA_df = p_data['LA'][day]['BVP'].copy()
                del p_data['LA'][day]['BVP']
                resample_ratio = round(len(LA_df))

                LA_df['ECG'] = signal.resample(p_data['LA'][day]['ECG']['ECG'], resample_ratio)
                del p_data['LA'][day]['ECG']
                #LA_df['BVP'] = signal.resample(p_data['LA'][day]['BVP']['BVP'], resample_ratio)
                LA_df['EDA'] = signal.resample(p_data['LA'][day]['EDA']['EDA'], resample_ratio)
                del p_data['LA'][day]['EDA']
                LA_df['TEMP'] = signal.resample(p_data['LA'][day]['TEMP']['Temp'], resample_ratio)
                del p_data['LA'][day]['TEMP']
                LA_df['BR'] = signal.resample(p_data['LA'][day]['BR']['breathing_rate'], resample_ratio)
                del p_data['LA'][day]['BR']
                LA_df['ACCx_hx'] = signal.resample(p_data['LA'][day]['ACC_hx']['Acceleration_X'], resample_ratio)
                del p_data['LA'][day]['ACC_hx']['Acceleration_X']
                LA_df['ACCy_hx'] = signal.resample(p_data['LA'][day]['ACC_hx']['Acceleration_Y'], resample_ratio)
                del p_data['LA'][day]['ACC_hx']['Acceleration_Y']
                LA_df['ACCz_hx'] = signal.resample(p_data['LA'][day]['ACC_hx']['Acceleration_Z'], resample_ratio)
                del p_data['LA'][day]['ACC_hx']['Acceleration_Z']   
                LA_df['ACCx_e4'] = signal.resample(p_data['LA'][day]['ACC_e4']['Acc_X'], resample_ratio)
                del p_data['LA'][day]['ACC_e4']['Acc_X']
                LA_df['ACCy_e4'] = signal.resample(p_data['LA'][day]['ACC_e4']['Acc_Y'], resample_ratio)
                del p_data['LA'][day]['ACC_e4']['Acc_Y']
                LA_df['ACCz_e4'] = signal.resample(p_data['LA'][day]['ACC_e4']['Acc_Z'], resample_ratio)
                del p_data['LA'][day]['ACC_e4']['Acc_Z']
                LA_df['rot_label'] = rot_anx_dict['LA']* np.ones(resample_ratio)
                LA_df['calib_label'] = np.zeros(resample_ratio)

                for column in columns_list:
                    if column == 'daily_check_in_date':
                        A = np.chararray(resample_ratio,itemsize=15,unicode=True)
                        A[:] = ((redcap_df.loc[redcap_df['participant'] == p]).iloc[day][column])
                        LA_df[column] = A
                    else:
                        LA_df[column] = np.ones(resample_ratio)* (redcap_df.loc[redcap_df['participant'] == p]).iloc[day][column] 
                
                print('Done with day ', day, 'date: ',(redcap_df.loc[redcap_df['participant'] == p]).iloc[day]['daily_check_in_date'],' of LA rotation for participant ', p)               
                p_df = pd.concat([p_df, LA_df], ignore_index=True, sort=False)
                #print('display LA_df after LA_df')
                #display(LA_df)
                

        for day in list(p_data['HA'].keys()):
            proceed2 = bool(len(p_data['HA'][day])) and bool(all_p_metadata[str(p)]['RedCap available'][1][day])
            if proceed2:
                    
                
                HA_df = p_data['HA'][day]['BVP'].copy()
                del p_data['HA'][day]['BVP']
                resample_ratio = round(len(HA_df))
                HA_df['ECG'] = signal.resample(p_data['HA'][day]['ECG']['ECG'], resample_ratio)
                del p_data['HA'][day]['ECG']
                #HA_df['BVP'] = signal.resample(p_data['HA'][day]['BVP']['BVP'], resample_ratio)

                HA_df['EDA'] = signal.resample(p_data['HA'][day]['EDA']['EDA'], resample_ratio)
                del p_data['HA'][day]['EDA']
                HA_df['TEMP'] = signal.resample(p_data['HA'][day]['TEMP']['Temp'], resample_ratio)
                del p_data['HA'][day]['TEMP']
                HA_df['BR'] = signal.resample(p_data['HA'][day]['BR']['breathing_rate'], resample_ratio)
                del p_data['HA'][day]['BR']
                HA_df['ACCx_hx'] = signal.resample(p_data['HA'][day]['ACC_hx']['Acceleration_X'], resample_ratio)
                del p_data['HA'][day]['ACC_hx']['Acceleration_X']
                HA_df['ACCy_hx'] = signal.resample(p_data['HA'][day]['ACC_hx']['Acceleration_Y'], resample_ratio)
                del p_data['HA'][day]['ACC_hx']['Acceleration_Y']
                HA_df['ACCz_hx'] = signal.resample(p_data['HA'][day]['ACC_hx']['Acceleration_Z'], resample_ratio)
                del p_data['HA'][day]['ACC_hx']['Acceleration_Z']
                HA_df['ACCx_e4'] = signal.resample(p_data['HA'][day]['ACC_e4']['Acc_X'], resample_ratio)
                del p_data['HA'][day]['ACC_e4']['Acc_X']
                HA_df['ACCy_e4'] = signal.resample(p_data['HA'][day]['ACC_e4']['Acc_Y'], resample_ratio)
                del p_data['HA'][day]['ACC_e4']['Acc_Y']
                HA_df['ACCz_e4'] = signal.resample(p_data['HA'][day]['ACC_e4']['Acc_Z'], resample_ratio)
                del p_data['HA'][day]['ACC_e4']['Acc_Z']
                HA_df['rot_label'] = rot_anx_dict['HA']* np.ones(resample_ratio) 
                HA_df['calib_label'] = np.zeros(resample_ratio)

                for column in columns_list:
                    if column == 'daily_check_in_date':
                        A = np.chararray(resample_ratio,itemsize=15,unicode=True)
                        A[:] = ((redcap_df.loc[redcap_df['participant'] == p]).iloc[day][column])
                        HA_df[column] = A
                        
                    else:
                        HA_df[column] = np.ones(resample_ratio)* (redcap_df.loc[redcap_df['participant'] == p]).iloc[day][column] 
                        
                print('Done with day ', day, 'date: ',(redcap_df.loc[redcap_df['participant'] == p]).iloc[day]['daily_check_in_date'],' of HA rotation for participant ', p)
                p_df = pd.concat([p_df, HA_df], ignore_index=True, sort=False)
                #print('display HA_df after HA_df')
                #display(HA_df)
                

        
        calib_df = p_data['calib']['BVP'].copy()
        del p_data['calib']['BVP']
        resample_ratio = round(len(calib_df))
        
        calib_df['ECG'] = signal.resample(p_data['calib']['ECG']['ECG'], resample_ratio)
        del p_data['calib']['ECG']
        #calib_df['BVP'] = signal.resample(p_data['calib']['BVP']['BVP'], resample_ratio)
        calib_df['EDA'] = signal.resample(p_data['calib']['EDA']['EDA'], resample_ratio)
        del p_data['calib']['EDA']
        calib_df['TEMP'] = signal.resample(p_data['calib']['TEMP']['Temp'], resample_ratio)
        del p_data['calib']['TEMP']
        calib_df['BR'] = signal.resample(p_data['calib']['BR']['breathing_rate'], resample_ratio)
        del p_data['calib']['BR']
        calib_df['ACCx_hx'] = signal.resample(p_data['calib']['ACC_hx']['Acceleration_X'], resample_ratio)
        del p_data['calib']['ACC_hx']['Acceleration_X']
        calib_df['ACCy_hx'] = signal.resample(p_data['calib']['ACC_hx']['Acceleration_Y'], resample_ratio)
        del p_data['calib']['ACC_hx']['Acceleration_Y']
        calib_df['ACCz_hx'] = signal.resample(p_data['calib']['ACC_hx']['Acceleration_Z'], resample_ratio)
        del p_data['calib']['ACC_hx']['Acceleration_Z']
        calib_df['ACCx_e4'] = signal.resample(p_data['calib']['ACC_e4']['Acc_X'], resample_ratio)
        del p_data['calib']['ACC_e4']['Acc_X']
        calib_df['ACCy_e4'] = signal.resample(p_data['calib']['ACC_e4']['Acc_Y'], resample_ratio)
        del p_data['calib']['ACC_e4']['Acc_Y']
        calib_df['ACCz_e4'] = signal.resample(p_data['calib']['ACC_e4']['Acc_Z'], resample_ratio)
        del p_data['calib']['ACC_e4']['Acc_Z']
        calib_df['rot_label'] = rot_anx_dict['calibration']* np.ones(resample_ratio)
        calib_df['calib_label'] = np.ones(resample_ratio) #complete later
        redcap_dfcalib = redcap_df # a cheat for now since i don't have redcap data for calibration
        columns_list = redcap_dfcalib.columns
        ############################################
        # redcap for calibration doesn't exist yet #
        ############################################

        for column in columns_list:
            #print(column)
            #print((redcap_dfcalib.loc[redcap_dfcalib['participant'] == p]).iloc[0][column]) # this is incorrect for now, replace with calibration data later
            if column == 'daily_check_in_date':
                A = np.chararray(resample_ratio,itemsize=15,unicode=True)
                A[:] = ((redcap_dfcalib.loc[redcap_dfcalib['participant'] == p]).iloc[0][column]) # this is incorrect for now, replace with calibration data later
                calib_df[column] = A
            else:
                calib_df[column] = np.ones(resample_ratio)* (redcap_dfcalib.loc[redcap_dfcalib['participant'] == p]).iloc[0][column] # this is incorrect for now, replace with calibration data later


        p_df = pd.concat([p_df, calib_df], ignore_index=True, sort=False)
        #print('display calib_df after calib')
        #display(calib_df)
        #display(calib_df)



        # save the data
        print('Pickling processed data for participant '+str(p)+ ' ...')
        with open(radwear_path +'p_'+str(p)+'.pkl', 'wb') as f:
            pickle.dump(p_df, f)
        print('data pickled...')
        
    else:
        print('processed data for participant '+str(p)+ ' is already available!')
        # load the data
        #with open(radwear_path +'p_'+str(p)+'.pkl', 'rb') as f:
        #    p_df = pickle.load(f)

# combining into 1 DF

In [ ]:
all_p_df = pd.DataFrame()

for p in list_of_participants:
    p_path = radwear_path 
    #p_df = pd.DataFrame()
    
    #check if file exist
    a = 'available.' if os.path.isfile(p_path + '/p_'+str(p)+'.pkl') else ' not available.'
    print('processed data for participant '+str(p)+ ' is '+ a)
    print('Loading...')
    with open(p_path +  '/p_'+str(p)+'.pkl', 'rb') as f:
        p_df = pickle.load(f)
    print ('participant '+str(p)+ ' data loaded.')
    print('-----------------------') 
    all_p_df = pd.concat([all_p_df, p_df], ignore_index=True, sort=False)
    display(p_df.sample(3))
    all_p_df.columns
print('pickling all participants file...')
all_p_df.to_pickle(radwear_path+'all_participants.pkl')
print('pickling completed. ')
print('DF for all participants is saved')